In [47]:
import pandas as pd

In [48]:
df = pd.read_csv("/home/alena/repos/scimagojr/scimagojr_full_2020-03-31.csv", sep=";")
df.drop(['Sourceid', 'Issn', 'SJR',
 'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)',
 'Ref. / Doc.', 'Country', 'Publisher', 'Coverage', 'Categories'], axis=1, inplace=True)

In [49]:
print(df.columns.values)
df.sample(10)

['Unnamed: 0' 'Rank' 'Title' 'Type' 'SJR Quartile' 'Area']


,Unnamed: 0,Rank,Title,Type,SJR Quartile,Area
3000,492,493,Proceedings - 2016 International Conference La...,conference and proceedings,-,Applied Mathematics
11257,117,118,Journal of the Urban Planning and Development ...,journal,Q2,Civil and Structural Engineering
70869,29,30,Child Language Teaching and Therapy,journal,Q3,Speech and Hearing
46880,89,90,2015 IEEE 31st International Conference on Sof...,conference and proceedings,-,Management Science and Operations Research
13895,174,175,MTD 2016 - Proceedings of the 2016 ACM Worksho...,conference and proceedings,-,Computer Networks and Communications
31521,38,39,Cancer Epidemiology,journal,Q2,Epidemiology
12205,93,94,"ASDAM 2016 - Conference Proceedings, 11th Inte...",conference and proceedings,-,Cognitive Neuroscience
60645,142,143,Inorganic and Nano-Metal Chemistry,journal,Q4,Physical and Theoretical Chemistry
5017,722,723,2015 54th Annual Conference of the Society of ...,conference and proceedings,-,Artificial Intelligence
14394,673,674,Proceedings - 2015 4th International Workshop ...,conference and proceedings,-,Computer Networks and Communications


In [50]:
df_cs = df.loc[df['Area'].str.contains('computer science', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
# df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False)]

In [51]:
s1 = pd.merge(df_cs, df_ling, how='inner', on=['Title'])

In [52]:
s1

,Unnamed: 0_x,Rank_x,Title,Type_x,SJR Quartile_x,Area_x,Unnamed: 0_y,Rank_y,Type_y,SJR Quartile_y,Area_y
0,53,54,Computer Assisted Language Learning,journal,Q1,Computer Science Applications,22,23,journal,Q1,Language and Linguistics
1,53,54,Computer Assisted Language Learning,journal,Q1,Computer Science Applications,26,27,journal,Q1,Linguistics and Language
2,68,69,Language Learning and Technology,journal,Q1,Computer Science Applications,28,29,journal,Q1,Language and Linguistics
3,68,69,Language Learning and Technology,journal,Q1,Computer Science Applications,31,32,journal,Q1,Linguistics and Language
4,80,81,ReCALL,journal,Q1,Computer Science Applications,34,35,journal,Q1,Language and Linguistics
5,80,81,ReCALL,journal,Q1,Computer Science Applications,40,41,journal,Q1,Linguistics and Language
6,210,211,Synthesis Lectures on Human Language Technologies,book series,Q2,Computer Science Applications,119,120,book series,Q1,Linguistics and Language
7,263,264,Speech Communication,journal,Q2,Computer Science Applications,127,128,journal,Q1,Language and Linguistics
8,263,264,Speech Communication,journal,Q2,Computer Science Applications,139,140,journal,Q1,Linguistics and Language
9,296,297,Computational Linguistics,journal,Q2,Computer Science Applications,137,138,journal,Q1,Language and Linguistics
